In [58]:
import os
import numpy as np
import pandas as pd
import joblib

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from catboost import CatBoostClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    roc_auc_score,
    confusion_matrix
)
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    KFold,
    GridSearchCV,
)
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE, SMOTENC, ADASYN
from imblearn.combine import SMOTEENN

In [59]:
def read_file(file_path):
    return pd.read_csv(file_path)

In [60]:
def data_split(df, test_size = None):
    X = df.drop('target', axis = 1)
    y = df['target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=42)

    return X_train, X_test, y_train, y_test

In [61]:
def data_augmentation(X_train, y_train):
    # aug = ADASYN(random_state=42)
    aug = SMOTEENN(random_state=42)
    X_train_resampled, y_train_resampled = aug.fit_resample(X_train, y_train)

    return X_train_resampled, y_train_resampled

In [62]:
def best_features(X_train, X_val, y_train):
    k_best_features = int(0.2 * X_train.shape[1])
    selector = SelectKBest(score_func=f_classif, k=k_best_features)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_val_selected = selector.transform(X_val)

    return X_train_selected, X_val_selected

In [63]:
def random_forest_learning_and_evaluation(x_train, y_train, x_val, y_val, param_grid):
    if param_grid:
        model = RandomForestClassifier(
            n_estimators=param_grid['n_estimators'], 
            max_depth=param_grid['max_depth'],
            min_samples_leaf=param_grid['min_samples_leaf'],
            min_samples_split=param_grid['min_samples_split'],
            random_state = 42
        )
    else:
        model = RandomForestClassifier(random_state=42)
        
    model.fit(x_train, y_train)

    y_pred = model.predict(x_val)
    print('Classification Report:\n', classification_report(y_val, y_pred))
    print('ROC-AUC Score:', roc_auc_score(y_val, y_pred))
    print('Confusion Matrix:\n', confusion_matrix(y_val, y_pred))

    return model

In [64]:
def parameter_tuning(x_train, y_train):
    param_grid = {
        'n_estimators': [10, 100],
        'max_depth': [6, 8, 10, 12],
        'min_samples_leaf': [8, 12, 18],
        'min_samples_split': [8, 16, 20],
    }

    grid_search = GridSearchCV(RandomForestClassifier(random_state = 42), param_grid, cv = 3, scoring = 'roc_auc')
    grid_search.fit(x_train, y_train)

    best_params = grid_search.best_params_
    print('Best Hyperparameters:', best_params)
    
    return best_params

In [65]:
def xgboost_learning_and_evaluation(x_train, y_train, x_val, y_val, param_grid):
    if param_grid:
        model = XGBClassifier(
            n_estimators=param_grid['n_estimators'], 
            max_depth=param_grid['max_depth'],
            learning_rate=param_grid['learning_rate'],
            random_state = 42,
        )
    else:
        model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss',)
        
    model.fit(x_train, y_train)

    y_pred = model.predict(x_val)
    print('Classification Report:\n', classification_report(y_val, y_pred))
    print('ROC-AUC Score:', roc_auc_score(y_val, y_pred))
    print('Confusion Matrix:\n', confusion_matrix(y_val, y_pred))

    return model

In [66]:
def xgboost_parameter_tuning(x_train, y_train):
    param_grid = {
        'n_estimators': [100, 200, 300, 400, 500, 600],
        'max_depth': [2, 3, 4, 5],
        'learning_rate': [0.01, 0.05, 0.1],
    }

    grid_search = GridSearchCV(
        XGBClassifier(random_state=42),
        param_grid, 
        cv=5, 
        scoring='f1_macro'
    )

    grid_search.fit(x_train, y_train)

    best_params = grid_search.best_params_
    print('Best Hyperparameters:', best_params)
    
    return best_params

In [67]:
def lightgbm_learning_and_evaluation(x_train, y_train, x_val, y_val, param_grid):
    if param_grid:
        model = LGBMClassifier(
            n_estimators=param_grid['n_estimators'], 
            max_depth=param_grid['max_depth'],
            learning_rate=param_grid['learning_rate'],
            subsample=param_grid['subsample'],
            colsample_bytree=param_grid['colsample_bytree'],
            random_state = 42,
            use_label_encoder=False,
            eval_metric='logloss',
        )
    else:
        model = LGBMClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss',)
        
    model.fit(x_train, y_train)

    y_pred = model.predict(x_val)
    print('Classification Report:\n', classification_report(y_val, y_pred))
    print('ROC-AUC Score:', roc_auc_score(y_val, y_pred))
    print('Confusion Matrix:\n', confusion_matrix(y_val, y_pred))

    return model

In [68]:
def lightgbm_parameter_tuning(x_train, y_train):
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0]
    }

    grid_search = GridSearchCV(
        LGBMClassifier(random_state=42),
        param_grid, 
        cv=3, 
        scoring='roc_auc'
    )

    grid_search.fit(x_train, y_train)

    best_params = grid_search.best_params_
    print('Best Hyperparameters:', best_params)
    
    return best_params

In [69]:
def adaboost_learning_and_evaluation(x_train, y_train, x_val, y_val, param_grid):
    if param_grid:
        model = AdaBoostClassifier(
            n_estimators=param_grid['n_estimators'], 
            learning_rate=param_grid['learning_rate'],
            base_estimator__max_depth=param_grid['base_estimator__max_depth'],
            random_state = 42,
        )
    else:
        model = AdaBoostClassifier(random_state=42)
        
    model.fit(x_train, y_train)

    y_pred = model.predict(x_val)
    print('Classification Report:\n', classification_report(y_val, y_pred))
    print('ROC-AUC Score:', roc_auc_score(y_val, y_pred))
    print('Confusion Matrix:\n', confusion_matrix(y_val, y_pred))

    return model

In [70]:
def adaboost_parameter_tuning(x_train, y_train):
    param_grid_ada = {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 1.0],
        'base_estimator__max_depth': [1, 2, 3]
    }

    base_estimator = DecisionTreeClassifier(random_state=42)
    grid_search = GridSearchCV(
        AdaBoostClassifier(base_estimator=base_estimator, random_state=42),
        param_grid_ada, 
        cv=3, 
        scoring='roc_auc'
    )

    grid_search.fit(x_train, y_train)

    best_params = grid_search.best_params_
    print('Best Hyperparameters:', best_params)
    
    return best_params

In [71]:
# PATH = r'D:\LGAimers\Hackerton\data\preprocessed_train_data_StandardScaler.csv'
# PATH = r'D:\LGAimers\Hackerton\data\preprocessed_train_data_RobustScaler.csv'
PATH = r'D:\LGAimers\Hackerton\data\preprocessed_train_data_Normalizer.csv'
df = read_file(PATH)

In [72]:
X_train, X_val, y_train, y_val = data_split(df, test_size = 0.3)

In [73]:
print(f'X_train.shape = {X_train.shape}')
print(f'X_val.shape = {X_val.shape}')
print(f'y_train.shape = {y_train.shape}')
print(f'y_val.shape = {y_val.shape}')

X_train.shape = (28354, 126)
X_val.shape = (12152, 126)
y_train.shape = (28354,)
y_val.shape = (12152,)


In [74]:
y_train.describe()

count    28354.000000
mean         0.940643
std          0.236295
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: target, dtype: float64

In [75]:
X_train_selected, X_val_selected = best_features(X_train=X_train, X_val=X_val, y_train=y_train)


In [76]:
X_train_resampled, y_train_resampled = data_augmentation(X_train=X_train_selected, y_train=y_train)

In [77]:
print(f'X_train_resampled.shape = {X_train_resampled.shape}')
print(f'y_train_resampled.shape = {y_train_resampled.shape}')

X_train_resampled.shape = (40593, 25)
y_train_resampled.shape = (40593,)


In [78]:
# best_params_adaboost = adaboost_parameter_tuning(X_train_resampled, y_train_resampled)

In [79]:
# grid_search_adaboost_model = adaboost_learning_and_evaluation(
#     x_train=X_train_resampled,
#     y_train=y_train_resampled,
#     x_val=X_val_selected,
#     y_val=y_val,
#     param_grid=best_params_adaboost,
# )

In [80]:
best_params_lgbm = lightgbm_parameter_tuning(X_train_resampled, y_train_resampled)

[LightGBM] [Info] Number of positive: 14126, number of negative: 12936
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001292 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6375
[LightGBM] [Info] Number of data points in the train set: 27062, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.521987 -> initscore=0.088003
[LightGBM] [Info] Start training from score 0.088003
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [81]:
grid_search_lgbm_model = lightgbm_learning_and_evaluation(
    x_train=X_train_resampled,
    y_train=y_train_resampled,
    x_val=X_val_selected,
    y_val=y_val,
    param_grid=best_params_lgbm,
)

[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Info] Number of positive: 21189, number of negative: 19404
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002338 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6375
[LightGBM] [Info] Number of data points in the train set: 40593, number of used features: 25
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.521987 -> initscore=0.088003
[LightGBM] [Info] Start training from score 0.088003
[LightGBM] [Warning] Unknown parameter: use_label_encoder
[LightGBM] [Warning] Unknown parameter: eval_metric
Classification Report:
               precision    recall  f1-score   support

           0       0.12      0.24      0.16       667
           1       0.95      0.90      0.93  

In [82]:
best_params_xgb = xgboost_parameter_tuning(X_train_resampled, y_train_resampled)

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 600}


In [83]:
grid_search_xgboost_model = xgboost_learning_and_evaluation(
    x_train=X_train_resampled,
    y_train=y_train_resampled,
    x_val=X_val_selected,
    y_val=y_val,
    param_grid=best_params_xgb,
)

Classification Report:
               precision    recall  f1-score   support

           0       0.12      0.23      0.16       667
           1       0.95      0.90      0.93     11485

    accuracy                           0.87     12152
   macro avg       0.54      0.56      0.54     12152
weighted avg       0.91      0.87      0.88     12152

ROC-AUC Score: 0.5644341521011371
Confusion Matrix:
 [[  151   516]
 [ 1120 10365]]


In [84]:
best_params_rf = parameter_tuning(X_train_resampled, y_train_resampled)

Best Hyperparameters: {'max_depth': 12, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}


In [85]:
grid_search_model = random_forest_learning_and_evaluation(
    x_train=X_train_resampled, 
    y_train=y_train_resampled, 
    x_val=X_val_selected, 
    y_val=y_val, 
    param_grid=best_params_rf,
)

Classification Report:
               precision    recall  f1-score   support

           0       0.11      0.38      0.17       667
           1       0.96      0.81      0.88     11485

    accuracy                           0.79     12152
   macro avg       0.53      0.60      0.52     12152
weighted avg       0.91      0.79      0.84     12152

ROC-AUC Score: 0.5977717497367989
Confusion Matrix:
 [[ 255  412]
 [2145 9340]]
